# Lemmatization of Latin inscriptions with CLTK

## Install CLTK

In [1]:
# do via a commandline, else it willf fail; https://github.com/fandoghpaas/fandogh-cli/issues/207

!pip install --upgrade --force-reinstall pip==9.0.3
!pip install fandogh-cli --disable-pip-version-check
#!pip install --upgrade setuptools

  Using cached https://files.pythonhosted.org/packages/ac/95/a05b56bb975efa78d3557efa36acaf9cf5d2fd0ee0062060493687432e03/pip-9.0.3-py2.py3-none-any.whl
  Found existing installation: pip 9.0.3
    Uninstalling pip-9.0.3:
      Successfully uninstalled pip-9.0.3
You are using pip version 9.0.3, however version 21.1.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install cltk

You are using pip version 9.0.3, however version 21.1.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
!pip install --upgrade pip
!pip install sddk

  Using cached https://files.pythonhosted.org/packages/47/ca/f0d790b6e18b3a6f3bd5e80c2ee4edbb5807286c21cdd0862ca933f751dd/pip-21.1.3-py3-none-any.whl
  Found existing installation: pip 9.0.3
    Uninstalling pip-9.0.3:
      Successfully uninstalled pip-9.0.3
  Using cached sddk-3.0-py3-none-any.whl (15 kB)
  Using cached matplotlib-3.4.2-cp37-cp37m-manylinux1_x86_64.whl (10.3 MB)
  Using cached Shapely-1.7.1-cp37-cp37m-manylinux1_x86_64.whl (1.0 MB)
  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)
  Using cached pyarrow-4.0.1-cp37-cp37m-manylinux2014_x86_64.whl (21.8 MB)
  Using cached geopandas-0.9.0-py2.py3-none-any.whl (994 kB)
  Using cached Fiona-1.8.20-cp37-cp37m-manylinux1_x86_64.whl (15.4 MB)
  Using cached pyproj-3.1.0-cp37-cp37m-manylinux2010_x86_64.whl (6.6 MB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Attempting unin

In [117]:
import sddk
import numpy as np
from cltk import NLP
import pandas as pd

## Load data

 Loading EDH dataset, and subsetting random 1000 inscriptions 

In [7]:
# read edh dataset
publicfolder = "b6b6afdb969d378b70929e86e58ad975"
EDH = sddk.read_file("EDH_text_cleaned_2021-01-21.json", "df", publicfolder)
EDH.head(5)

reading file located in a public folder


,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,...,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military
0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"[{'name': 'Noniae P.f. Optatae', 'cognomen': '...",2.7 cm,...,D M Noniae P f Optatae et C Iulio Artemoni par...,Dis Manibus Noniae Publi filiae Optatae et Cai...,Dis Manibus Noniae Publi filiae Optatae et Cai...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"[{'nomen': 'Sextius', 'praenomen': 'C.', 'pers...",NaN,...,C Sextius Paris qui vixit annis LXX,Caius Sextius Paris qui vixit annis LXX,Caius Sextius Paris qui vixit annis LXX,"Via Nomentana, S. Alessandro, Kirche",1937,NaN,NaN,NaN,NaN,NaN
2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"[{'nomen': 'Mummius+', 'gender': 'male', 'prae...",(12) cm,...,ummio isenna Xv,Publio Mummio Publi filio Galeria Sisennae Rut...,Publio Mummio Publi filio Galeria Sisennae Rut...,NaN,before 1975,"Sevilla, Privatbesitz",NaN,NaN,NaN,NaN
3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,[{'name': '[---?]AV(?)S(?)[---]L(?)L(?)A M. Po...,18 cm,...,AVSLLA M Porci Nigri ser dominae Veneri aram p...,AVS LLA Marci Porci Nigri serva dominae Veneri...,AVS LLA Marci Porci Nigri serva dominae Veneri...,NaN,before 1979,"Carcabuey, Grupo Escolar",names of pagan deities,NaN,NaN,NaN
4,Feraudi,epitaph,NaN,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,NaN,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"[{'person_id': '1', 'name': '[---]l. Successus...",NaN,...,l Successus L l Irenaeus C l Tyches unt renti f,libertus Successus Luci libertus Irenaeus Cai ...,libertus Successus Luci libertus Irenaeus Cai ...,Via Cupa (ehem. Vigna Nardi),NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Creating random sample with 1000 records

sample = EDH.sample(n=1000)
sample


,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,...,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military
46483,Cowey,building/dedicatory inscription?,NaN,0200,"AE 2001, 1283.; R.S.O. Tomlin - M.W.C. Hassall...",checked with photo,(9.7) cm,]C[ / / ]A[,{},(4.1) cm,...,CA,C A,C A,Römisch-keltischer Tempel,before 1986,"Harlow, Museum",NaN,NaN,NaN,NaN
48745,Feraudi,epitaph,4 cm,0270,"IDR 3, 4, 111.",no image,NaN,D [ ] / VAL V[ ] / AEL [,"[{'nomen': 'Valerius/a*', 'person_id': '1', 'n...",NaN,...,D Val V Ael,Dis Manibus Valerius V Aelius,Dis Manibus Valerius V Aelius,"Sântămărie, ehemalige reformierte Kirche, seku...",NaN,NaN,NaN,NaN,NaN,NaN
19190,Scheithauer,NaN,NaN,NaN,"AE 1955, 0142.",provisional,NaN,MATRI DEVM / AVG SAC / [ ] GRATTIVS MVSTIO / L...,{},NaN,...,Matri Deum Aug sac Grattius Mustiolus aed II v...,Matri Deum Augustae sacrum Grattius Mustiolus ...,Matri Deum Augustae sacrum Grattius Mustiolus ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43046,Gräf,epitaph,NaN,0200,"CIL 03, 08239.; ILJug 1446.",no image,NaN,D M / NANEA PRVD / ENTIS FILIA / POSVIT CO / N...,"[{'name': 'Nanea Prudentis filia', 'cognomen':...",NaN,...,D M Nanea Prudentis filia posuit coniugi suo c...,Dis Manibus Nanea Prudentis filia posuit coniu...,Dis Manibus Nanea Prudentis filia posuit coniu...,"Xërxë / Zrze, bei, Ura e Shenjtë / Švanjskog M...",NaN,NaN,NaN,NaN,NaN,data available
62151,Feraudi,epitaph,3 cm,0400,"CIL 03, 08843; Zeichnung.; E. Marin u. a., Sal...",checked with photo,(15.5) cm,]ONAR C[ ] / [ ] ALEXAND[ ] / [ ]T AN I,"[{'person_id': '1', 'age: years': '1', 'nomen'...",11 cm,...,ONAR C Alexand t an I,ONAR C Alexand vixit annum I,ONAR C Alexand vixit annum I,1-279 etc. bleibend (111),1886,"Split, Arh. Muz.",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24124,Gräf,epitaph,NaN,0100,"CIL 02 (2. Aufl.) 14, 00106.; CIL 02, 03658.",no image,NaN,L FABIVS PRO / CVLVS AN LX / VITORIA OMVL / LI...,"[{'praenomen': 'L.', 'name': 'L. Fabius Procul...",NaN,...,L Fabius Proculus an LX Vitoria Omullina an XX...,Lucius Fabius Proculus annorum LX Vitoria Omul...,Lucius Fabius Proculus annorum LX Vitoria Omul...,NaN,NaN,lost,NaN,NaN,NaN,NaN
31987,Scheithauer,NaN,NaN,NaN,"AE 1901, 0002.",provisional,NaN,IMP CAESARI L SEPTIMIO SEVE / RO PIO RT AVG ET...,{},NaN,...,Imp Caesari L Septimio Severo Pio rt Aug et Im...,Imperatori Caesari Lucio Septimio Severo Pio P...,Imperatori Caesari Lucio Septimio Severo Pio P...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63011,Feraudi,epitaph,NaN,0150,"CIL 03, 02096; Zeichnung.; CIL 03, 02096 add. ...",checked with photo,NaN,P FVNDANIVS PHILOLOGVS / AVGVSTAL T F I SIB ET...,"[{'cognomen': 'Philologus', 'status': 'Augusta...",NaN,...,P Fundanius Philologus Augustal t f i sib et F...,Publius Fundanius Philologus Augustalis titulu...,Publius Fundanius Philologus Augustalis titulu...,1-279 etc. bleibend (111),NaN,NaN,NaN,NaN,data available,NaN
40071,Garcia,votive inscription,NaN,0250,"ILB (2. Aufl.) 045; Zeichnung.; CIL 13, 03613.",checked with drawing,NaN,IN H D D / D I M AX[ ] / VERVS [ ] / Q VETI[ ]...,"[{'name': 'Ax[ius?] Verus', 'nomen': 'Axius+?'...",NaN,...,In h d d D I M Ax Verus Q Veti et Probinus Ver...,In honorem domus divinae Deo Invicto Mithrae A...,In honorem domus divinae Deo Invicto Mithrae A...,NaN,NaN,lost,names of pagan deities,NaN,NaN,NaN


In [42]:
sample.to_json(r'EDH_random_sample_1000.json')

## Run NLP pipeline with NLP()

In [33]:
cltk_nlp = NLP(language="lat")

‎𐤀 CLTK version '1.0.17'.
Pipeline for language 'Latin' (ISO: 'lat'): `LatinNormalizeProcess`, `LatinStanzaProcess`, `LatinEmbeddingsProcess`, `StopsProcess`, `LatinNERProcess`, `LatinLexiconProcess`.


### Test on one inscription

In [84]:
test = "Matri Deum Augustae sacrum Grattius Mustiolus"

In [85]:
# characters
len(test)

45

In [87]:
# words
len(test.split())

6

In [89]:
%time
cltk_doc = cltk_nlp.analyze(text=test)

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 30 µs
CLTK message: This part of the CLTK depends upon the Stanza NLP library.
CLTK message: Allow download of Stanza models to ``/home/ucloud/stanza_resources/la/tokenize/ittb.pt``? [Y/n] 
y


2021-07-20 15:06:56 INFO: Downloading these customized packages for language: la (Latin)...
| Processor | Package |
-----------------------
| tokenize  | ittb    |
| pos       | ittb    |
| lemma     | ittb    |
| depparse  | ittb    |
| pretrain  | ittb    |

2021-07-20 15:07:55 INFO: Finished downloading models and saved to /home/ucloud/stanza_resources.


CLTK message: This part of the CLTK depends upon word embedding models from the Fasttext project.
Do you want to download file 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.la.vec' to '/home/ucloud/cltk_data/lat/embeddings/fasttext/wiki.la.vec'? [Y/n] 
y


100%|██████████| 364M/364M [00:28<00:00, 12.6MiB/s]    


This part of the CLTK depends upon models from the CLTK project.
Do you want to download 'https://github.com/cltk/lat_models_cltk' to '~/cltk_data/lat'? [Y/n] 
 y
Please respond with 'yes' or 'no' (or 'y' or 'n').
Do you want to download 'https://github.com/cltk/lat_models_cltk' to '~/cltk_data/lat'? [Y/n] 
y
This part of the CLTK depends upon Lewis's *An Elementary Latin Dictionary* (1890).
Do you want to download this? [Y/n] 
y


In [91]:
# We can now inspect the result
print(type(cltk_doc))
# All accessors
print([x for x in dir(cltk_doc) if not x.startswith("__")])

<class 'cltk.core.data_types.Doc'>
['_get_words_attribute', 'embeddings', 'embeddings_model', 'language', 'lemmata', 'morphosyntactic_features', 'normalized_text', 'pipeline', 'pos', 'raw', 'sentence_embeddings', 'sentences', 'sentences_strings', 'sentences_tokens', 'stanza_doc', 'stems', 'tokens', 'tokens_stops_filtered', 'words']


In [103]:
# Several of the more useful, compared with the original
print(cltk_doc.raw)

# List of tokens
print(cltk_doc.tokens[:20])

Matri Deum Augustae sacrum Grattius Mustiolus
['Matri', 'Deum', 'Augustae', 'sacrum', 'Grattius', 'Mustiolus']


In [93]:
# List of lemmas
print(cltk_doc.lemmata[:20])

['mater', 'meus', 'mugusta', 'sacer', 'mraeter', 'Mustiolus']


In [94]:
# Basic part-of-speech info
print(cltk_doc.pos[:20])

['NOUN', 'PROPN', 'PROPN', 'NOUN', 'VERB', 'NOUN']


In [104]:
print(cltk_doc.morphosyntactic_features)

[{Case: [dative], Degree: [positive], Gender: [feminine], Number: [singular]}, {Case: [accusative], Degree: [positive], Gender: [masculine], Number: [singular]}, {Case: [genitive], Degree: [positive], Gender: [masculine], Number: [singular]}, {Case: [accusative], Degree: [positive], Gender: [neuter], Number: [singular]}, {Aspect: [perfective], Case: [nominative], Degree: [positive], Gender: [masculine], Number: [singular], Tense: [past], VerbForm: [participle], Voice: [passive]}, {Case: [nominative], Degree: [positive], Gender: [masculine], Number: [singular]}]


None


### Testing with multiple inscriptions

In [133]:
inscriptions = sample["clean_text_interpretive_word"]
inscriptions

46483                                                  C A
48745                        Dis Manibus Valerius V Aelius
19190    Matri Deum Augustae sacrum Grattius Mustiolus ...
43046    Dis Manibus Nanea Prudentis filia posuit coniu...
62151                         ONAR C Alexand vixit annum I
                               ...                        
24124    Lucius Fabius Proculus annorum LX Vitoria Omul...
31987    Imperatori Caesari Lucio Septimio Severo Pio P...
63011    Publius Fundanius Philologus Augustalis titulu...
40071    In honorem domus divinae Deo Invicto Mithrae A...
55058                                     Sever coniugi MO
Name: clean_text_interpretive_word, Length: 1000, dtype: object

In [134]:
inscriptions = pd.DataFrame(inscriptions)

In [123]:
inscriptions

,clean_text_interpretive_word,tokenized
46483,C A,"[C, A]"
48745,Dis Manibus Valerius V Aelius,"[Dis, Manibus, Valerius, V, Aelius]"
19190,Matri Deum Augustae sacrum Grattius Mustiolus ...,"[Matri, Deum, Augustae, sacrum, Grattius, Must..."
43046,Dis Manibus Nanea Prudentis filia posuit coniu...,"[Dis, Manibus, Nanea, Prudentis, filia, posuit..."
62151,ONAR C Alexand vixit annum I,"[ONAR, C, Alexand, vixit, annum, I]"
...,...,...
24124,Lucius Fabius Proculus annorum LX Vitoria Omul...,"[Lucius, Fabius, Proculus, annorum, LX, Vitori..."
31987,Imperatori Caesari Lucio Septimio Severo Pio P...,"[Imperatori, Caesari, Lucio, Septimio, Severo,..."
63011,Publius Fundanius Philologus Augustalis titulu...,"[Publius, Fundanius, Philologus, Augustalis, t..."
40071,In honorem domus divinae Deo Invicto Mithrae A...,"[In, honorem, domus, divinae, Deo, Invicto, Mi..."


In [127]:


%time 

cltk_ins = cltk_nlp.analyze(text=str(inscriptions['clean_text_interpretive_word']))

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 14.1 µs


!!!!! NOTE: I need to find a way how to lemmatize the text in dataframe, using the CLTK.

In [129]:
print(type(cltk_ins))
print([x for x in dir(cltk_ins) if not x.startswith("__")])


<class 'cltk.core.data_types.Doc'>
['_get_words_attribute', 'embeddings', 'embeddings_model', 'language', 'lemmata', 'morphosyntactic_features', 'normalized_text', 'pipeline', 'pos', 'raw', 'sentence_embeddings', 'sentences', 'sentences_strings', 'sentences_tokens', 'stanza_doc', 'stems', 'tokens', 'tokens_stops_filtered', 'words']


In [130]:
print(cltk_ins.tokens[:20])
print(cltk_ins.lemmata[:20])
print(cltk_ins.pos[:20])

['46483', 'C', 'A', '48745', 'Dis', 'Manibus', 'Valerius', 'V', 'Aelius', '19190', 'Matri', 'Deum', 'Augustae', 'sacrum', 'Grattius', 'Mustiolus', '.', '..', '43046', 'Dis']
['num._arab.', 'num._rom.', 'ex', 'num._arab.', 'Dis', 'manus', 'Valerius', 'num._rom.', 'melius', 'num._arab.', 'mater', 'meus', 'mugusta', 'sacer', 'mraeter', 'Mustiolus', '.', '..', 'num._arab.', 'Dis']
['NUM', 'NUM', 'ADP', 'NUM', 'NOUN', 'NOUN', 'PROPN', 'NUM', 'PROPN', 'NUM', 'NOUN', 'PROPN', 'PROPN', 'ADJ', 'VERB', 'NOUN', 'PUNCT', 'NOUN', 'NUM', 'NOUN']


## NLTK

In [18]:
!pip install nltk

In [78]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/ucloud/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### One inscription

In [81]:
sample_text = "Dis Manibus Valerius V Aelius"
word_list = nltk.word_tokenize(sample_text)
print(word_list)

['Dis', 'Manibus', 'Valerius', 'V', 'Aelius']


In [82]:
  
lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
print(lemmatized_output)

Dis Manibus Valerius V Aelius


### Multiple inscriptions

#### Tokenization

In [135]:
inscriptions["tokenized"] = inscriptions.apply(lambda row: nltk.word_tokenize(row['clean_text_interpretive_word']), axis=1)

In [136]:
inscriptions

,clean_text_interpretive_word,tokenized
46483,C A,"[C, A]"
48745,Dis Manibus Valerius V Aelius,"[Dis, Manibus, Valerius, V, Aelius]"
19190,Matri Deum Augustae sacrum Grattius Mustiolus ...,"[Matri, Deum, Augustae, sacrum, Grattius, Must..."
43046,Dis Manibus Nanea Prudentis filia posuit coniu...,"[Dis, Manibus, Nanea, Prudentis, filia, posuit..."
62151,ONAR C Alexand vixit annum I,"[ONAR, C, Alexand, vixit, annum, I]"
...,...,...
24124,Lucius Fabius Proculus annorum LX Vitoria Omul...,"[Lucius, Fabius, Proculus, annorum, LX, Vitori..."
31987,Imperatori Caesari Lucio Septimio Severo Pio P...,"[Imperatori, Caesari, Lucio, Septimio, Severo,..."
63011,Publius Fundanius Philologus Augustalis titulu...,"[Publius, Fundanius, Philologus, Augustalis, t..."
40071,In honorem domus divinae Deo Invicto Mithrae A...,"[In, honorem, domus, divinae, Deo, Invicto, Mi..."


#### Lemmatization

In [1]:
# There is no NLTK model for Latin inscriptions, but I was testing with wordnet if it even words on dataframes

In [156]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()


def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in word_tokenize(text)]


inscriptions['text_lemmatized'] = inscriptions.clean_text_interpretive_word.apply(lemmatize_text)
inscriptions

[nltk_data] Downloading package wordnet to /home/ucloud/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,clean_text_interpretive_word,tokenized,text_lemmatized
46483,C A,"[C, A]","[C, A]"
48745,Dis Manibus Valerius V Aelius,"[Dis, Manibus, Valerius, V, Aelius]","[Dis, Manibus, Valerius, V, Aelius]"
19190,Matri Deum Augustae sacrum Grattius Mustiolus ...,"[Matri, Deum, Augustae, sacrum, Grattius, Must...","[Matri, Deum, Augustae, sacrum, Grattius, Must..."
43046,Dis Manibus Nanea Prudentis filia posuit coniu...,"[Dis, Manibus, Nanea, Prudentis, filia, posuit...","[Dis, Manibus, Nanea, Prudentis, filia, posuit..."
62151,ONAR C Alexand vixit annum I,"[ONAR, C, Alexand, vixit, annum, I]","[ONAR, C, Alexand, vixit, annum, I]"
...,...,...,...
24124,Lucius Fabius Proculus annorum LX Vitoria Omul...,"[Lucius, Fabius, Proculus, annorum, LX, Vitori...","[Lucius, Fabius, Proculus, annorum, LX, Vitori..."
31987,Imperatori Caesari Lucio Septimio Severo Pio P...,"[Imperatori, Caesari, Lucio, Septimio, Severo,...","[Imperatori, Caesari, Lucio, Septimio, Severo,..."
63011,Publius Fundanius Philologus Augustalis titulu...,"[Publius, Fundanius, Philologus, Augustalis, t...","[Publius, Fundanius, Philologus, Augustalis, t..."
40071,In honorem domus divinae Deo Invicto Mithrae A...,"[In, honorem, domus, divinae, Deo, Invicto, Mi...","[In, honorem, domus, divinae, Deo, Invicto, Mi..."
